In [1]:
'''

export SUBJECTS_DIR=/data/sheng/MEG
export SUBJECT=case_0000

### 1. THIS SNIPET IS FOR REGULAR SUBJECTSS
mkheadsurf -subjid $SUBJECT  # create high-res head model for coreg

# 2. schaefer 200 
cd /data/sheng/MEG
./Schaefer_script.sh


## 3. Shrink inner bem model if needed
### THIS SNIPET IS FOR REGULAR SUBJECTSS BY x 
export SHRINKAGE=4

sudo chmod -R +777 $SUBJECTS_DIR/$SUBJECT
cd $SUBJECTS_DIR/$SUBJECT
sudo mkdir $SUBJECTS_DIR/$SUBJECT/bem/new
sudo chmod -R +777 $SUBJECTS_DIR/$SUBJECT/bem/new
cd $SUBJECTS_DIR/$SUBJECT/bem/new
cp $SUBJECTS_DIR/$SUBJECT/mri/T1.mgz $SUBJECTS_DIR/$SUBJECT/bem/new/T1-bem.mgz
cd $SUBJECTS_DIR/$SUBJECT/bem/new/
mri_watershed -useSRAS -atlas -shk_br_surf $SHRINKAGE bembrain -surf bem T1-bem.mgz ws_brain.mgz
cd $SUBJECTS_DIR/$SUBJECT/bem/
ln -sf  new/bem_inner_skull_surface inner_skull.surf
ln -sf  new/bem_outer_skull_surface outer_skull.surf
ln -sf  new/bem_outer_skin_surface outer_skin.surf




# Executed on 5.12.2024

export SUBJECTS_DIR=/data/sheng/MEG
export SUBJECT=case_0000
sudo chmod -R +777 $SUBJECTS_DIR/$SUBJECT
cd $SUBJECTS_DIR/$SUBJECT
recon-all -all -subjid $SUBJECT
mne watershed_bem --overwrite -s $SUBJECT
mkheadsurf -subjid $SUBJECT  # create high-res head model for coreg
cd /data/sheng/MEG
./Schaefer_script.sh


sudo chmod -R +777 $SUBJECTS_DIR/$SUBJECT
cd $SUBJECTS_DIR/$SUBJECT
sudo mkdir $SUBJECTS_DIR/$SUBJECT/bem/new
sudo chmod -R +777 $SUBJECTS_DIR/$SUBJECT/bem/new
cd $SUBJECTS_DIR/$SUBJECT/bem/new
cp $SUBJECTS_DIR/$SUBJECT/mri/T1.mgz $SUBJECTS_DIR/$SUBJECT/bem/new/T1-bem.mgz
cd $SUBJECTS_DIR/$SUBJECT/bem/new/
mri_watershed -useSRAS -atlas -shk_br_surf 6 bembrain -surf bem T1-bem.mgz ws_brain.mgz

# after visualizing the bem models, symbolic link
cd $SUBJECTS_DIR/$SUBJECT/bem/
ln -sf  new/bem_inner_skull_surface inner_skull.surf
ln -sf  new/bem_outer_skull_surface outer_skull.surf
ln -sf  new/bem_outer_skin_surface outer_skin.surf




'''

############################################################
## fixing inner_skull outside of outer skull problem 
## - based on Judith's slack post, Follow Judith's instruction
############################################################
'''


recon-all -skullstrip -clean-bm -gcut -subjid $SUBJECT
recon-all -autorecon2-cp -autorecon3 -subjid $SUBJECT


# from here
export SUBJECTS_DIR=/data/sheng/MEG
export SUBJECT=case_0000
sudo chmod -R +777 $SUBJECTS_DIR/$SUBJECT
cd $SUBJECTS_DIR/$SUBJECT
sudo mkdir $SUBJECTS_DIR/$SUBJECT/bem/new
sudo chmod -R +777 $SUBJECTS_DIR/$SUBJECT/bem/new
cd $SUBJECTS_DIR/$SUBJECT/bem/new
cp $SUBJECTS_DIR/$SUBJECT/mri/T1.mgz $SUBJECTS_DIR/$SUBJECT/bem/new/T1-bem.mgz
cd $SUBJECTS_DIR/$SUBJECT/bem/new/
mri_watershed -useSRAS -atlas -shk_br_surf 3 bembrain -surf bem T1-bem.mgz ws_brain.mgz

# after visualizing the bem models, symbolic link
cd $SUBJECTS_DIR/$SUBJECT/bem/
ln -sf  new/bem_inner_skull_surface inner_skull.surf
ln -sf  new/bem_outer_skull_surface outer_skull.surf
ln -sf  new/bem_outer_skin_surface outer_skin.surf

'''




'\n\n\nrecon-all -skullstrip -clean-bm -gcut -subjid $SUBJECT\nrecon-all -autorecon2-cp -autorecon3 -subjid $SUBJECT\n\n\n# from here\nexport SUBJECTS_DIR=/data/sheng/MEG\nexport SUBJECT=case_0118\nsudo chmod -R +777 $SUBJECTS_DIR/$SUBJECT\ncd $SUBJECTS_DIR/$SUBJECT\nsudo mkdir $SUBJECTS_DIR/$SUBJECT/bem/new\nsudo chmod -R +777 $SUBJECTS_DIR/$SUBJECT/bem/new\ncd $SUBJECTS_DIR/$SUBJECT/bem/new\ncp $SUBJECTS_DIR/$SUBJECT/mri/T1.mgz $SUBJECTS_DIR/$SUBJECT/bem/new/T1-bem.mgz\ncd $SUBJECTS_DIR/$SUBJECT/bem/new/\nmri_watershed -useSRAS -atlas -shk_br_surf 3 bembrain -surf bem T1-bem.mgz ws_brain.mgz\n\n# after visualizing the bem models, symbolic link\ncd $SUBJECTS_DIR/$SUBJECT/bem/\nln -sf  new/bem_inner_skull_surface inner_skull.surf\nln -sf  new/bem_outer_skull_surface outer_skull.surf\nln -sf  new/bem_outer_skin_surface outer_skin.surf\n\n'

In [ ]:
#%% import modules
import os
import os.path as op
import numpy as np
from mayavi import mlab
import shutil
import mne
from preprocessor.filehandling import get_files
from mne.datasets import sample
import time
import math
#print(__doc__)

#%% set folder and subject
# guay_parker_spont_rsa_1_chpi_raw.fif
subjects_dir    = '/data/sheng/MEG/'
Subj            = 'case_0000'
dir1            = subjects_dir + Subj + '/'
filepath, filename = get_files(op.join(subjects_dir, Subj, ''), incl_strings = ['spont_rsa', 'tsss.fif'],  ftype='.fif')

# save the coreg file:
for iFile in filename:
    print(iFile)

In [ ]:
# !!!! assign set num corresponding to the spont dataset you wnat to process !!!!
setname         = 'set1' 

coreg_dir       = dir1 + 'freesurfer/coreg' 
coreg_file      = coreg_dir + '/' + Subj  + '_' + setname + '_coreg_trans.fif'  # e.g., case_0000_set1_coreg_trans.fif
#trans_file      = subjects_dir + 'Subj + coreg_file
trans_file      = coreg_file
src_file        = subjects_dir + Subj + '/bem/' + Subj + '-5-src.fif'
mriFile         = dir1 + 'mri/T1.mgz'
sol_file        = subjects_dir + Subj + '/bem/' + Subj + '_' + setname + '-bem-sol.fif'
print('Next step: coregistration ********************************************')
print('Save coreg file as: ' + coreg_file)

In [ ]:
mne.utils.set_config("SUBJECTS_DIR", subjects_dir, set_env=True)
mne.gui.coregistration(subject = Subj, subjects_dir=subjects_dir)

In [ ]:
########################################################################
#%% visualize bem surfaces                                            ##
########################################################################
orientation = 'axial'            # can be 'coronal', 'axial', or 'sagittal'
surf        = 'pial'            # 'pial' or 'white' 
mne.viz.plot_bem(Subj, subjects_dir = subjects_dir, brain_surfaces=surf,orientation=orientation, mri=mriFile)
mne.viz.plot_bem(Subj, subjects_dir = subjects_dir, src=src_file, brain_surfaces=surf,orientation=orientation, mri=mriFile)

In [75]:
#%% get list of raw files
filenames = [f for f in os.listdir(subjects_dir + Subj) if ('fif' in f and 'ica_cleaned' in f)]
print(np.array(filenames))

['guay_parker_spont_rsa_1_chpi_raw_tsss_filt_ica_cleaned.fif']


In [20]:
########################################################################
# Forward model --- Felix's script, obsolete now 11.2024 !!!!
# pick the appropirate fif file from above to make fwd solution       ##
########################################################################
'''
fwd_dir         = dir1 + 'source/ForwardOperators/' 
#sol_file        = subjects_dir + '/' + Subj + '/bem/sub-14023131-5120-bem-sol.fif'

filename    = 'guay_parker_spont_rsa_1_chpi_raw_tsss_filt_ica_cleaned.fif'
raw_file    = dir1 + filename
#date        = filename[-10:-4]
set_no      = 1    #filename[6:12] # use index from 1 onward
info        = mne.io.read_raw_fif(raw_file, on_split_missing='warn').info
fwd         = mne.make_forward_solution(info,trans_file,src_file,sol_file,meg=True,eeg=False,n_jobs=10)

## Write fwd solution
fwd_file        = fwd_dir + Subj + '_set' + str(set_no) + '_py-fwd.fif'        
os.makedirs(fwd_dir,exist_ok=True)                
mne.write_forward_solution(fwd_file, fwd,overwrite=True) 
'''

"\nfilename    = 'guay_parker_spont_rsa_1_chpi_raw_tsss_filt_ica_cleaned.fif'\nraw_file    = dir1 + filename\n#date        = filename[-10:-4]\nset_no      = 1    #filename[6:12] # use index from 1 onward\ninfo        = mne.io.read_raw_fif(raw_file, on_split_missing='warn').info\nfwd         = mne.make_forward_solution(info,trans_file,src_file,sol_file,meg=True,eeg=False,n_jobs=10)\n\n## Write fwd solution\nfwd_file        = fwd_dir + Subj + '_set' + str(set_no) + '_py-fwd.fif'        \nos.makedirs(fwd_dir,exist_ok=True)                \nmne.write_forward_solution(fwd_file, fwd,overwrite=True) \n"

In [ ]:
### don't run this #######################################################
#cond = [0.3]                         # 1-layer (can't be used for EEG)
'''
cond = [0.3, 0.006, 0.3]              # 3-layer (can be used for EEG)
model           = mne.bem.make_bem_model(subject=Subj,subjects_dir=subjects_dir, conductivity=cond)
solution        = mne.bem.make_bem_solution(model)
mne.bem.write_bem_solution(sol_file, solution)
'''

In [ ]:
print('subj_dir:\t' + dir1)
print('mriFile:\t' + mriFile)
print('trans_file:\t' + trans_file)
print('src_file:\t' + src_file)
print('sol_file:\t' + sol_file)